<a href="https://colab.research.google.com/github/RonBartov/Erasure_Probabilities_and_Error_Correction/blob/main/erasure_probabilities_and_error_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Erasure Probabilities and Error Correction Simulations in Channel Decoding**

## **Background**
The following assignment focuses on understanding and simulating the behavior of error correction systems in binary erasure channels (BEC). The tasks involve computing erasure probabilities, implementing encoding and decoding functions, and analyzing system performance under various configurations.

**Key objectives include:**

${\circ}$ Developing and simulating functions for error probabilities and bit-channel transforms.

${\circ}$ Understanding how erasure probabilities evolve in a BEC under specific conditions.

${\circ}$ Analyzing decoding success rates, frame error rates (FER), and multi-channel communication.

# HERE, ADDITIONAL INFORMATION WILL BE ADDED
- NOTES
- DIVIDING THE ASSIGNMENT INTO DIFFERENT PARTS

# IN SEPARATE TEXT BOX, THE THEORY AND FORMULAS FOR CALCULATING PROBABILITIES WILL BE ADDED

# Import Necessary Libraries

In [ ]:
import numpy as np
import math

# Part 1: Computing Erasure Probabilities

This part will include the following:

${\circ}$ Implementing functions for computing and plotting erasure probabilities, based on the theoretical background.

${\circ}$ Analysis of results and observed phenomena.

In [ ]:
def verify_power_of_two(n: int) -> None:
    """
    Verifies whether a given integer is a power of two.

    Args:
        n (int): The integer to check.

    Raises:
        ValueError: If `n` is not a positive power of two.
    """
    if n <= 0 or (n & (n - 1)) != 0:
        raise ValueError(f"Input must be a positive power of two, but got {n}.")

In [ ]:
def permutation_block(layer: int, input_array: np.ndarray) -> np.ndarray:
    """
    Permutes the elements of an input numpy array based on the specified layer value.

    The function rearranges the elements of the input array according to the following rules:
    - For indices j = 0, ..., 2^(l+1)-1:
      - If j < 2^l: the element at index j is moved to index 2j.
      - Otherwise (j >= 2^l): the element at index j is moved to index 2(j - 2^l) + 1.

    Args:
        layer (int): The layer value 'l' that determines the permutation logic.
        input_array (np.ndarray): The numpy array of floats to be permuted.

    Returns:
        np.ndarray: A new numpy array with elements permuted according to the given rules.

    Raises:
        ValueError: If the length of the input array is not equal to 2^(l+1).
    """
    n = len(input_array)
    expected_length = 2 ** (layer + 1)

    if n != expected_length:
        raise ValueError(f"Input array length must be {expected_length}, but got {n}.")

    permuted_array = np.zeros_like(input_array)  # Initialize a new array of the same size with zeros
    threshold = 2 ** layer

    for j in range(n):
        if j < threshold:
            new_index = 2 * j
        else:
            new_index = 2 * (j - threshold) + 1

        permuted_array[new_index] = input_array[j]

    return permuted_array

In [ ]:
def array_permutation_by_block(layer: int, input_array: np.ndarray) -> np.ndarray:
    """
    Permutes the elements of an input numpy array by blocks using the permutation_block function.

    The input array is divided into blocks of size 2^(l+1), and the permutation_block function
    is applied to each block. The permuted blocks are then concatenated to form the final output array.

    Args:
        layer (int): The layer value 'l' that determines the block size and permutation logic.
        input_array (np.ndarray): The numpy array of floats to be permuted by blocks.

    Returns:
        np.ndarray: A new numpy array with elements permuted by blocks according to the given rules.

    Raises:
        ValueError: If the size of the input array is not a multiple of the block size.
    """
    n = len(input_array)
    block_size = 2 ** (layer + 1)

    if n % block_size != 0:
        raise ValueError(f"Input array size must be a multiple of the block size {block_size}, but got {n}.")

    num_blocks = n // block_size
    permuted_array = []

    for i in range(num_blocks):
        block_start = i * block_size
        block_end = block_start + block_size
        block = input_array[block_start:block_end]
        permuted_block = permutation_block(layer, block)
        permuted_array.append(permuted_block)

    return np.concatenate(permuted_array)

In [ ]:
def effective_bit_channels_in_bcd(error_probabilities: np.ndarray) -> np.ndarray:
    """
    Calculates the effective bit channels for a bit channel decoder based on input error probabilities.

    The function computes the effective bit channel probabilities for the decoder output using the input
    error probabilities. The calculation is based on even and odd indexed elements as follows:
    - For even indices j = 0, 2, ..., n-2:
      x_out[j] = 1 - [1 - x_in[j]] * [1 - x_in[j+1]]
    - For odd indices k = 1, 3, ..., n-1:
      x_out[k] = x_in[k-1] * x_in[k]

    Args:
        error_probabilities (np.ndarray): A numpy array of floats representing the input error probabilities.

    Returns:
        np.ndarray: A numpy array of floats representing the effective bit channel probabilities for the decoder output.

    Raises:
        ValueError: If the length of the input array is not a power of 2.
    """
    n = len(error_probabilities)

    if not (n and (n & (n - 1)) == 0):
        raise ValueError("Input array length must be a power of 2, but got {n}.")

    output_probabilities = np.zeros_like(error_probabilities)

    for j in range(0, n, 2):
        output_probabilities[j] = 1 - (1 - error_probabilities[j]) * (1 - error_probabilities[j + 1])

    for k in range(1, n, 2):
        output_probabilities[k] = error_probabilities[k - 1] * error_probabilities[k]

    return output_probabilities

In [ ]:
def erasure_probability_recursive_u_transform(n: int, epsilon: int) -> np.ndarray:
    """
    Computes the erasure probabilities of bit channels recursively using the U-transform.

    Args:
        n (int): The block size, which must be a power of 2. Represents the total number of bit channels.
        epsilon (int): The initial erasure probability of the input channels.

    Returns:
        np.ndarray: A NumPy array of size `n` containing the computed erasure probabilities
                    for each bit channel.

    Raises:
        ValueError: If `n` is not a power of 2 or is less than 2.

    Note:
        - The function uses recursive splitting to compute the probabilities, halving the
          size of the problem in each recursive step.
        - The `effective_bit_channels_in_bcd` function is used to calculate the erasure
          probabilities after combining the split channels.
        - The `permutation_block` function applies a specific permutation to the input
          probabilities, based on the current layer of recursion.

    """
    verify_power_of_two(n)

    if n == 2:
        input_probabilities = np.array([epsilon, epsilon])
        output_probabilities = effective_bit_channels_in_bcd(input_probabilities)
        return output_probabilities
    else:
        layer = int(math.log2(n)) - 1
        input_probabilities_half_size = erasure_probability_recursive_u_transform(n//2, epsilon)
        input_probabilities = np.concatenate((input_probabilities_half_size, input_probabilities_half_size))
        input_probabilities_after_permutation = permutation_block(layer, input_probabilities)
        output_probabilities = effective_bit_channels_in_bcd(input_probabilities)
        return output_probabilities

In [ ]:
n = 2 ** 10
u = erasure_probability_recursive_u_transform(n, 0.5)
print(len(u))

1024
